In [1]:
train_projects = [ # total: 13 point clouds (including bbox splitting)
    # {"proj_name": "2023-08-28_FW_EingangBauing.FwfProj", "bboxes" : []},                    # building
    {"proj_name": "2024-03-22_FW_Koenigshuegel.FwfProj", "bboxes" : []},                      # building
    # {"proj_name": "2024-04-05_FW_Westbahnhof_01.FwfProj", "bboxes" : []},                   # tunnel-bridge
    {"proj_name": "2024-04-05_FW_Westbahnhof_02.FwfProj", "bboxes" : []},                     # tunnel-bridge
    {"proj_name": "2024-04-05_FW_Westbahnhof_03.FwfProj", "bboxes" : []},                     # tunnel-bridge
    {"proj_name": "2024-04-05_FW_Westbahnhof_04.FwfProj", "bboxes" : []},                     # tunnel-bridge
    {"proj_name": "2024-04-05_FW_Westbahnhof_05.FwfProj", "bboxes" : []},                     # building
    {"proj_name": "2024-05-10_FW_RWTH_Zentrum_01.FwfProj", "bboxes" : []},                    # building
    {"proj_name": "2024-07-31_FW_Bruecke_Koenigstr.FwfProj", "bboxes" : [0,2]},               # bridge (steel+stone)
    # {"proj_name": "2024-07-31_FW_Bruecke_Turmstr.FwfProj", "bboxes" : []},                  # bridge (concrete/steel)
    {"proj_name": "2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr.FwfProj", "bboxes" : []},   # bridge (concrete)
    {"proj_name": "2024-08-02_FW_Bruecke_Deltourserb.FwfProj", "bboxes" : []},                # bridge (concrete)
    {"proj_name": "2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj", "bboxes" : [1]},             # bridge (concrete, steel, brick)
    {"proj_name": "2024-08-02_FW_Bruecke_RotheErde.FwfProj", "bboxes" : []},                  # bridge (steel)
    {"proj_name": "2024-08-02_FW_Bruecke_Rottstrasse.FwfProj", "bboxes" : []},                # bridge (concrete)
]
val_projects = [ # total: 5 point clouds (including bbox splitting)
    {"proj_name": "2023-08-28_FW_EingangBauing.FwfProj", "bboxes" : []},                    # building
    {"proj_name": "2024-04-05_FW_Westbahnhof_01.FwfProj", "bboxes" : []},                   # tunnel-bridge
    {"proj_name": "2024-07-31_FW_Bruecke_Koenigstr.FwfProj", "bboxes" : [1]},               # bridge (steel+stone)
    {"proj_name": "2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj", "bboxes" : [0]},           # bridge (concrete, steel, brick)
    {"proj_name": "2024-07-31_FW_Bruecke_Turmstr.FwfProj", "bboxes" : []},                  # bridge (concrete/steel)


]

In [2]:
import numpy as np
class BBox:
    """Bounding box using the CloudCompare synthax
    """
    def __init__(self, orientation:np.ndarray, width: np.ndarray) -> None:
        # an easy way to retrieve these values is by using the cross-section tool in cloudcompare
        # so cross-section -> advanced -> orientation -> to clipboard
        self.orientation = orientation
        # from the same menu copy and paste the values under 'width' 
        self.width = width
        
    def cutout(self, xyz:np.ndarray) -> np.ndarray:
            # Inverse of the orientation matrix (rotation and translation)
            orientation_inv = np.linalg.inv(self.orientation)
            
            # Transform points into the bbox's local coordinate system
            transformed_xyz = (orientation_inv @ np.vstack((xyz.T, np.ones(xyz.shape[0])))).T
            
            # Get the half-dimensions of the bbox (since width is the full length)
            half_width = self.width / 2
            
            # Check if points lie within the bbox in local coordinates
            inside_bbox = (
                (np.abs(transformed_xyz[:, 0]) <= half_width[0]) &
                (np.abs(transformed_xyz[:, 1]) <= half_width[1]) &
                (np.abs(transformed_xyz[:, 2]) <= half_width[2])
            )
            
            # Return the indices of the points inside the bbox
            return np.where(inside_bbox)[0]


In [3]:
from utils.pointcloud import grid_subsample
import numpy as np
import torch

def grid_subsample_simple(xyz:np.ndarray, voxel_size:float, device='cuda'):
    """
    Wrapper using numpy arrays
    """
    sub = grid_subsample(torch.Tensor(xyz).to(device=device), voxel_size) # n -> m number of points
    sub['points'] = sub['points'].to(dtype=torch.float64).detach().cpu().numpy()
    sub['inv_inds'] = sub['inv_inds'].detach().cpu().numpy()
    return sub

In [4]:
from typing import Callable, Tuple, List
import colorsys

class Transform():
    def __init__(self, fn:Callable = lambda x: x):
        self.fn = fn

    def __call__(self, x):
        return self.fn(x)

class TransZRotation(Transform):
    def __init__(self, limits:Tuple = (0, 2*np.pi)):
        def fn(x):
            r = np.random.uniform(*limits)
            rot_matrix = np.array([[np.cos(r), -np.sin(r), 0],
                                   [np.sin(r),  np.cos(r), 0],
                                   [0,          0,         1]])
            return x @ rot_matrix
            
        super().__init__(fn)


class TransScaling(Transform):
    def __init__(self, limits: Tuple = (0.85, 1.15)):
        def fn(x):
            scale_factor = np.random.uniform(*limits)
            scale_matrix = np.diag([scale_factor, scale_factor, scale_factor])  # Uniform scaling
            return x @ scale_matrix
        
        super().__init__(fn)

class TransGaussianNoise(Transform):
    def __init__(self, mean: float = 0.0, std: float = 0.01):
        def fn(x:np.ndarray):
            noise = np.random.normal(loc=mean, scale=std, size=x.shape)
            return x + noise

        super().__init__(fn)

class TransGammaCorrection(Transform):
    def __init__(self, limits: Tuple[float, float] = (0.8, 1.2)):
        def fn(x):
            gamma = np.random.uniform(*limits)
            return np.clip(x ** gamma, 0, 1) # TODO: Check order of normalization and augmentation !!!
        super().__init__(fn)

class TransSignalScaling(Transform):
    def __init__(self,limits:Tuple[float,float] = (0.95,1.05)):
        def fn(x:np.ndarray):
            # waveform shape is neibors, 32
            scale_factors = np.random.uniform(*limits, size=(x.shape[0],1))
            return x * scale_factors
        
        super().__init__(fn)
        
class TransFeatureDropout(Transform):
    def __init__(self, dropout_prob: float = 0.2):

        def fn(x):
            mask = np.random.binomial(1, 1 - dropout_prob, size=x.shape[1])
            return x * mask  
        
        super().__init__(fn)

class TransStandardize(Transform):
    def __init__(self, mean, std):
        def fn(x:np.ndarray):
            x -= mean
            x /= std
            return x
        super().__init__(fn)

class TransformsList():
    def __init__(self,
                 transforms:List[Transform]):
        self.transforms = transforms

    def __call__(self, x):
        for t in self.transforms:
            x = t(x)
        return x



In [5]:
# res_idx = 56
# proj = train_ds.projects[0]
# neibs = proj['neibors'][res_idx]
# rgb = proj['rgb'][neibs]


# transform = TransformsList([
#     #  TransGammaCorrection(),
#     TransStandardize(mean=np.array([[0.29689665, 0.3428666,  0.190237]]),std=np.array([[0.21558372, 0.23351644, 0.21213871]])),
#     TransGaussianNoise(0,0.02),
#     TransFeatureDropout(dropout_prob=0.1)                         
# ])

# print(rgb.shape, rgb.mean(), rgb.std())
# rgb = transform(rgb)
# print(rgb.shape, rgb.mean(), rgb.std())


In [6]:
from plyfile import PlyData, PlyElement
import pandas as pd
import numpy as np
from glob import glob
import json
import os
from omegaconf import OmegaConf
from scipy.spatial import KDTree
from torch.utils.data import Dataset, DataLoader
from jakteristics import compute_features
from typing import List, Optional, Dict
import torch

from utils.pointcloud import grid_subsample

class FwfDataset(Dataset):
    def __init__(self,
                 proj_search_pattern,
                 proj_query_list,
                 return_fields_input = [
                    'xyz',
                    'rgb',
                    'riegl_feats',
                    'normals',
                    'incAngles',
                    'distanceFromScanner',
                ],
                 label_names = None,
                 return_waveform = True,
                 query_grid_size = None,
                 subsample_on_gpu = True,
                 transforms_dict: Dict[str, TransformsList] = dict(
                     xyz = TransformsList([
                         TransZRotation(),
                         TransScaling(),
                         TransGaussianNoise(0,0.002)
                     ]),
                     rgb = TransformsList([
                        #  TransGammaCorrection(),
                        #  TransStandardize(mean=np.array([[0.29689665, 0.3428666,  0.190237]]),std=np.array([[0.21558372, 0.23351644, 0.21213871]])),
                         TransGaussianNoise(0,0.02),
                         TransFeatureDropout(dropout_prob=0.1)                         
                     ]),
                     wfm = TransformsList([
                        # TransStandardize(mean=358.35934, std=623.0141),
                        TransSignalScaling(),
                     ]),
                     normals = TransformsList([
                         TransGaussianNoise(0,0.05)
                     ]),
                     incAngles = TransformsList([
                         TransGaussianNoise(0,0.05),
                         TransStandardize(mean = 2.07, std = 0.38),
                     ]),
                     distanceFromScanner = TransformsList([
                         TransGaussianNoise(0,0.1),
                         TransStandardize(mean = 12.13, std = 12.20),
                     ]),
                    #  riegl_feats = TransformsList([
                    #      TransStandardize(mean=np.array([[-6.90191949, 25.16398933, 26.45952891,  1.,  1.03636612]]), std= np.array([[2.32590898, 2.98518547, 929.71399545, 1., 0.22651793]]))
                    #  ])
                 )
                 ):
        
        super(Dataset,self).__init__()
        
        self.proj_query_list = proj_query_list
        self.return_fields_input = return_fields_input
        self.return_waveform = return_waveform
        self.transforms_dict = transforms_dict

        self.projects = list()
        
        # keep track of point cloud sizes
        self.proj_lens = []

        
        counter = 0
        # go through search pattern
        for fwf_prof_fp in glob(proj_search_pattern):
            proj_name = os.path.basename(fwf_prof_fp)
            
            # only get projects if they are on the project list
            if proj_name not in [p['proj_name'] for p in self.proj_query_list]:
                continue
            else:
                bbox_queries = [p['bboxes'] for p in self.proj_query_list if p['proj_name']==proj_name][0]

                
            print(f"Loading '{proj_name}'; Bounding box IDs = {bbox_queries if len(bbox_queries) else 'default'}")

            # load the data
            pcd = pd.DataFrame(PlyData.read(list(glob(os.path.join(fwf_prof_fp,'labeled','*pointcloud.ply')))[0]).elements[0].data)
            wfm = np.load(list(glob(os.path.join(fwf_prof_fp,'labeled','*waveform.npy')))[0]).astype(np.float32)
            meta = json.load(open(list(glob(os.path.join(fwf_prof_fp,'labeled','*metadata.json')))[0],"r"))
            
            # field names (constants)
            riegl_feat_names = [ 'riegl_reflectance','riegl_amplitude', 'riegl_deviation', 'riegl_targetIndex','riegl_targetCount']
            # label_names = ['labels_0', 'labels_1', 'labels_2', 'labels_3']
            self.label_names:Optional[List[str]] = label_names
            
            
            # convert to numpy
            riegl_feats = pcd[riegl_feat_names].to_numpy()
            rgb = pcd[['Red','Green','Blue']].to_numpy()
            
            # normalize the data
            # FIXME: Statistics calculated on first point cloud only
            wfm = (wfm.astype(np.float32) - 358.35934) / 623.0141 # - mean / std

            riegl_feats -= np.array([[-6.90191949, 25.16398933, 26.45952891,  1.        ,  1.03636612]]) # - means
            riegl_feats /= np.array([[  2.32590898, 2.98518547, 929.71399545, 1., 0.22651793]]) # /std
            
            rgb -= np.array([[0.29689665, 0.3428666,  0.190237]]) # means
            rgb /= np.array([[0.21558372, 0.23351644, 0.21213871]]) # std          
            

            
            # get scan positions
            if 'scanId=000' in meta['scan_positions'].keys():
                sop = np.array([meta['scan_positions'][f'scanId={si:03}']['sop'] for si in range(len(meta['scan_positions']))])
            else:
                # handle case where only one scan position is in the metadata and it's f.s.r. labeled 'scanId=001' instead of 'scanID=000'
                sop = np.array([meta['scan_positions']['scanId=001']['sop']])

            # get full xyz
            xyz_defaultBbox = pcd[['x','y','z']].to_numpy()       

                
            # save project as dict
            if len(bbox_queries)==0:
                # handle default case
                sub = grid_subsample_simple(xyz_defaultBbox,query_grid_size, 'cuda' if subsample_on_gpu else 'cpu')
                kd_tree = KDTree(xyz_defaultBbox)
                _, sub_ids = kd_tree.query(sub['points'])
                self.projects.append(dict(
                    proj_name=f"{proj_name}::defaultBbox",
                    xyz = xyz_defaultBbox,
                    wfm = wfm,
                    sop = sop,
                    rgb = rgb,
                    riegl_feats = riegl_feats,
                    labels = pcd[self.label_names].to_numpy()[sub_ids], # labels need to be subsampeld 
                    sop_ids = pcd['scan_id'].to_numpy(),
                    kd_tree = kd_tree,
                    xyz_sub = sub['points'],
                    sub_inv = sub['inv_inds']
                ))
                self.proj_lens.append(sub['points'].shape[0])
            else:
                # handle region bboxes case
                for bbox_i in bbox_queries:
                    bbox_meta = meta['bboxes'][f'bboxId={bbox_i:03}']
                    bbox = BBox(orientation=np.array(bbox_meta['orientation']), width=np.array(bbox_meta['width']))
                    subcloud_mask=bbox.cutout(xyz_defaultBbox)
                    xyz_masked = xyz_defaultBbox[subcloud_mask]
                    sub = grid_subsample_simple(xyz_masked,query_grid_size, 'cuda' if subsample_on_gpu else 'cpu')
                    kd_tree = KDTree(xyz_masked)
                    _, sub_ids = kd_tree.query(sub['points'])
                    self.projects.append(dict(
                        proj_name=f"{proj_name}::bboxId={bbox_i:03}",
                        xyz = xyz_masked,
                        wfm = wfm[subcloud_mask],
                        sop = sop,
                        rgb = rgb[subcloud_mask],
                        riegl_feats = riegl_feats[subcloud_mask],
                        labels = pcd[self.label_names].to_numpy()[subcloud_mask][sub_ids], # labels need to be subsampeld
                        sop_ids = pcd['scan_id'].to_numpy()[subcloud_mask],
                        kd_tree = KDTree(xyz_masked),
                        xyz_sub = sub['points'],
                        sub_inv = sub['inv_inds']
                    ))
                    self.proj_lens.append(sub['points'].shape[0])
                    
            # calculate the cumulative sum of the point cloud sizes

            if counter >= 3:
                continue
                break
            else:
                counter += 1
        self.proj_lens_cumsum = np.cumsum(self.proj_lens)
    
    def compute_neibors_knn(self, k:int):
        for proj in self.projects:
            print(f"Computing neibors for '{proj['proj_name']}' @ k={k}")
            dists, neib_ids = proj['kd_tree'].query(proj['xyz_sub'], k)
            proj['neibors'] = neib_ids
    

   
    def compute_normals_knn(self):
        for proj in self.projects:
            k = proj['neibors'].shape[1]
            print(f"Computing normals for '{proj['proj_name']}' @ k={k}")
            neibs_xyz = proj['xyz'][proj['neibors']]

            means = neibs_xyz.mean(axis=1, keepdims=True)
            neibs_xyz -= means
            cov = (neibs_xyz.transpose([0,2,1]) @ neibs_xyz) / (k-1)
            eigenvals, eigenvecs = np.linalg.eigh(cov)
            # get non-flipped normals
            normals = eigenvecs[:, :, 0]

            # upsample normals to full resolution
            normals = normals[proj['sub_inv']]
            
            # move all points to scanner CS
            points_origin_scanPos = proj['sop'][proj['sop_ids']][:,:3,3]
            xyz_scannerCs = proj['xyz'] - points_origin_scanPos
            signs = np.sign(np.squeeze(xyz_scannerCs[:,None,:] @ normals [:,:,None])) * -1
            normals *= signs[:,None]
            
            proj['normals'] = normals




            
            
    # def compute_normals_spherical_subsample(self, r = 0.08, voxel_size = 0.03, device='cuda'):
        
    #     for proj in self.projects:
    #         print(f"Computing normals for '{proj['proj_name']}' @ r={r:.3f}m and voxel_size = {voxel_size:.3f}")
    #         print("\tDEBUG: subsampling")
    #         sub = grid_subsample(torch.Tensor(proj['xyz']).to(device=device), voxel_size) # n -> m number of points
    #         sub['points'] = sub['points'].to(dtype=torch.float64).detach().cpu().numpy()
    #         sub['inv_inds'] = sub['inv_inds'].detach().cpu().numpy()
    #         _,sub_ids = proj['kd_tree'].query(sub['points'])
    #         print("\tDEBUG: computing feats")
    #         sub_normals = compute_features(sub['points'],search_radius=0.05,feature_names=['nx','ny','nz'])

    #         # flip the normals
    #         print("\tDEBUG: flipping the subsampled normals")
    #         sub_xyz_scannerCs = proj['xyz'][sub_ids] - proj['sop'][proj['sop_ids']][:,3,:3][sub_ids]
    #         sub_signs = np.sign(np.squeeze(sub_xyz_scannerCs[:,None,:] @ sub_normals[:,:,None]))*-1
    #         sub_normals *= sub_signs[:,None]

    #         print("\tDEBUG: reprojecting")
    #         proj['normals'] = sub_normals[sub['inv_inds']]



    
    
    # def compute_normals_spherical(self, r = 0.08):
    #     for proj in self.projects:
    #         print(f"Computing normals for '{proj['proj_name']}' @ r={r:.3f}m")
            
    #         # get non-oriented normals
    #         normals = compute_features(proj['xyz'],search_radius=0.05,feature_names=['nx','ny','nz'])
    #         # orient the normals to corresponding scanner position
    #         xyz_scannerCs = proj['xyz'] - proj['sop'][proj['sop_ids']][:,3,:3]
    #         signs = np.sign(np.squeeze(xyz_scannerCs[:,None,:] @ normals [:,:,None])) * -1
    #         normals *= signs[:,None]
            
            
            
    def compute_incAngles(self):
        for proj in self.projects:
            print(f"Computing incidence angles for '{proj['proj_name']}'")
            xyz_scannerCs = proj['xyz'] - proj['sop'][proj['sop_ids']][:,:3,3]
            proj['incAngles']= np.arccos(np.squeeze((proj['normals'][:,None,:] @ xyz_scannerCs[...,None])) / \
                (np.linalg.norm(xyz_scannerCs,axis=-1) * np.linalg.norm(proj['normals'],axis=-1)))
            proj['distanceFromScanner'] = np.linalg.norm(xyz_scannerCs, axis=-1)
            
            # reshape by adding an additional axis
            proj['incAngles'] = proj['incAngles'][:,None]
            proj['distanceFromScanner'] = proj['distanceFromScanner'][:,None]
            
    
    def __getitem__(self, index):
        # get proj index first
        proj_idx = np.argwhere(index<self.proj_lens_cumsum)[0][0]
        size_prev = self.proj_lens_cumsum[proj_idx-1] if proj_idx > 0 else 0
        residual_idx = index - size_prev
        
        # get neibors of point at index
        neibs = self.projects[proj_idx]['neibors'][residual_idx]
        
        

        # TODO: remove the default option (instead putting the fields as a default argument list to constructor)

            # features_neibors = np.concatenate([
            #     self.projects[proj_idx]['xyz'][neibs],
            #     self.projects[proj_idx]['rgb'][neibs],
            #     self.projects[proj_idx]['riegl_feats'][neibs],
            #     self.projects[proj_idx]['normals'][neibs],
            #     self.projects[proj_idx]['incAngles'][neibs],
            #     self.projects[proj_idx]['distanceFromScanner'][neibs],
            # ], axis=-1).astype(np.float32),
            # features_point = np.concatenate([
            #     self.projects[proj_idx]['xyz'][residual_idx][None,:],
            #     self.projects[proj_idx]['rgb'][residual_idx][None,:],
            #     self.projects[proj_idx]['riegl_feats'][residual_idx][None,:],
            #     self.projects[proj_idx]['normals'][residual_idx][None,:],
            #     self.projects[proj_idx]['incAngles'][residual_idx][None,:],
            #     self.projects[proj_idx]['distanceFromScanner'][residual_idx][None,:],
            # ],axis=-1).astype(np.float32)


        return_dict = dict(
            features_neibors = np.concatenate(
                [
                    self.transforms_dict[f](self.projects[proj_idx][f][neibs])
                    if f in self.transforms_dict else self.projects[proj_idx][f][neibs]
                    for f in self.return_fields_input
                ],
                axis=-1
            ).astype(np.float32),
            features_point = np.concatenate(
                [
                    self.transforms_dict[f](self.projects[proj_idx][f][residual_idx][None,:])
                    if f in self.transforms_dict else self.projects[proj_idx][f][residual_idx][None,:]
                    for f in self.return_fields_input
                ],
                axis=-1
            ).astype(np.float32)
        )

        # add waveforms and labels
        return_dict.update(dict(
            wfm_neibors = self.projects[proj_idx]['wfm'][neibs] if self.return_waveform else None,
            wfm_point = self.projects[proj_idx]['wfm'][residual_idx][None,:] if self.return_waveform else None,
            labels = self.projects[proj_idx]['labels'][residual_idx],
        )) # type:ignore
        

        
        return return_dict
        
    
    def __len__(self):
        return np.sum([p['xyz_sub'].shape[0] for p in self.projects])
        




In [7]:
from torch import nn

class FGFeatNetwork (nn.Module):
    def __init__(self,
                 num_input_feats,
                 label_structure = {'labels_0':3, 'labels_1':10, 'labels_2':12, 'labels_3':18},
                 dropout_prob = 0.3,
                 global_constraint = True,
                 ):
        super(FGFeatNetwork, self).__init__()
        
        self.label_structure = label_structure
        self.global_constraint = global_constraint
        # Pointwise feats
        self.mlp1 = nn.Sequential(
            nn.Linear(num_input_feats, 64), nn.ReLU(), nn.Dropout(p=dropout_prob),
            nn.Linear(64, 128), nn.ReLU(), nn.Dropout(p=dropout_prob),
            nn.Linear(128, 128), nn.ReLU(), nn.Dropout(p=dropout_prob),
        )
        self.wf_conv = nn.Sequential(
            # FIXME: check conv and maxpool kernel sizes and strides
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3), nn.RReLU(),nn.BatchNorm1d(16), nn.MaxPool1d(2), # activations 32 -> 16
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3), nn.RReLU(),nn.BatchNorm1d(32), nn.MaxPool1d(2),# activations 16 -> 8
            nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3), nn.RReLU(),nn.BatchNorm1d(32), nn.MaxPool1d(2),# activations 8 -> 4
            # after concat. activation shape = 4 * 64 = 256
        )
        # TODO: Try out inverse bottleneck?
        # TODO: Network size might be an overkill / unfeasable for the task
        
        # MLP after concat with WFM feats
        self.mlp2 = nn.Sequential(
            nn.Linear(192, 512), nn.ReLU(), nn.Dropout(p=dropout_prob), 
            nn.Linear(512, 512), nn.ReLU(), nn.Dropout(p=dropout_prob),
        )
        
        # decoder
        self.mlp3 = nn.Sequential(
            nn.Linear(704, 512), nn.ReLU(), nn.Dropout(p=dropout_prob),  # joined shape (point + neibors)
            nn.Linear(512, 256), nn.ReLU(), nn.Dropout(p=dropout_prob),
            nn.Linear(256, 128), nn.ReLU(), nn.Dropout(p=dropout_prob),
        )
        
        # classifier
        self.classifier = nn.ModuleDict({k:nn.Linear(128,v) for k,v in self.label_structure.items()})
        
    def forward(self, x):
        # handle neibor features
        pw_feats_neib = self.mlp1(x['features_neibors'])
        batch, neibor, signal_length = x['wfm_neibors'].shape
        wf_feats_neib = self.wf_conv(x['wfm_neibors'].view((batch * neibor, 1, signal_length))) # batch, neibor, signal_length -> batch * neibor, 1,  signal_length
        wf_feats_neib = wf_feats_neib.view((batch, neibor,-1))
        joined_feats_neib = torch.cat([pw_feats_neib, wf_feats_neib], dim=-1)
        
        # handle the point features
        pw_feats_point = self.mlp1(x['features_point'])
        batch, neibor, signal_length = x['wfm_point'].shape
        wf_feats_point = self.wf_conv(x['wfm_point'].view((batch * neibor, 1, signal_length))) ## batch, 1, signal_length -> batch , 1,  signal_length
        wf_feats_point = wf_feats_point.view((batch, neibor,-1))
        joined_feats_point = torch.cat([pw_feats_point, wf_feats_point], dim=-1)        
        
        # continue processing neibor feats
        
        joined_feats_neib = self.mlp2(joined_feats_neib)
        joined_feats_neib = torch.max(joined_feats_neib, dim=1)[0]
        
        # join neibor with skip connection to point features
        global_feat = torch.cat([joined_feats_neib, torch.squeeze(joined_feats_point)],dim=-1)
        if self.global_constraint:
            global_feat = global_feat / (torch.norm(global_feat, p=2, dim=1, keepdim=True)+1e-6)

        global_feat = self.mlp3(global_feat)
        
        result = {k: self.classifier[k](global_feat) for k in self.classifier.keys()}
        
        
        
        return result

        
        

In [8]:
from torch.nn import CrossEntropyLoss
from tqdm import tqdm


criterion = CrossEntropyLoss()
device='cuda'
label_names = [
    'labels_0',
    # 'labels_1',
    # 'labels_2',
    # 'labels_3',
]
query_grid_size = 0.1
num_neib_normals = 20
num_neib_featextraction = 96



# create Dataset
proj_search_pattern = r"D:\Projekte\GIA_220412_PCS\02_Datasets\FullWaveForm\full_waveform_software\FullWaveformParse_mass\output\FWF_Aachen_labeled\*.FwfProj" # work pc
# proj_search_pattern = "../../02_Datasets/FWF_Aachen_labeled/*.FwfProj" # home pc
train_ds = FwfDataset(
    proj_search_pattern=proj_search_pattern, 
    proj_query_list=train_projects, 
    query_grid_size=query_grid_size,
    label_names = label_names,
    return_waveform = True,
    subsample_on_gpu = True
)
val_ds = FwfDataset(
    proj_search_pattern=proj_search_pattern, 
    proj_query_list=val_projects, 
    query_grid_size=query_grid_size,
    label_names = label_names,
    return_waveform = True,
    subsample_on_gpu = True
)

# rest computation
for ds in [train_ds, val_ds]:
    ds.compute_neibors_knn(k=num_neib_normals)
    ds.compute_normals_knn()
    ds.compute_incAngles()
    ds.compute_neibors_knn(k=num_neib_featextraction)
    







Loading '2024-03-22_FW_Koenigshuegel.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_02.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_03.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_04.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_05.FwfProj'; Bounding box IDs = default
Loading '2024-05-10_FW_RWTH_Zentrum_01.FwfProj'; Bounding box IDs = default
Loading '2024-07-31_FW_Bruecke_Koenigstr.FwfProj'; Bounding box IDs = [0, 2]
Loading '2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr.FwfProj'; Bounding box IDs = default
Loading '2024-08-02_FW_Bruecke_Deltourserb.FwfProj'; Bounding box IDs = default
Loading '2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj'; Bounding box IDs = [1]
Loading '2024-08-02_FW_Bruecke_RotheErde.FwfProj'; Bounding box IDs = default
Loading '2024-08-02_FW_Bruecke_Rottstrasse.FwfProj'; Bounding box IDs = default
Loading '2023-08-28_FW_EingangBauing.FwfProj'; Bounding box IDs 

In [9]:
# create model
model = FGFeatNetwork(
    num_input_feats = train_ds[0]['features_neibors'].shape[-1],
    label_structure = {
        "labels_0":3,
        # "labels_1":8,
        # "labels_2":18,
        # "labels_3":18
    },
    global_constraint=True
    ).to(device=device)

# create dataloader

batch_size=64

train_dl = DataLoader(train_ds, batch_size=batch_size)
val_dl = DataLoader(val_ds, batch_size=batch_size)

In [10]:
from torch.optim import Adam

weight_decay = 1e-4
max_epochs = 100


optim = Adam(params=model.parameters(), weight_decay=weight_decay)

for epoch in range(max_epochs):

    epoch_train_loss = []
    # train
    for batch in tqdm(train_dl, desc=f"{'Training':<15}", leave=True):
        optim.zero_grad()
        # put batch on device
        for k, v in batch.items():
            batch[k] = v.to(device=device)

        # forward pass
        result = model(batch)
        
        # agregate loss on all levels
        loss = torch.tensor(0.).to(device=device)
        for k in result.keys():
            output = result[k]
            gt = batch['labels'][:, train_ds.label_names.index(k)] # type:ignore
            loss += criterion(output, gt)
        
        loss.backward()
        optim.step()
        epoch_train_loss.append(loss.item())
    # validate    
    epoch_val_loss = []
    with torch.no_grad():
        for batch in tqdm(val_dl, desc=f"{'Validation':<15}", leave=True):
            # put batch on device
            for k, v in batch.items():
                batch[k] = v.to(device=device)

            # forward pass
            result = model(batch)
            
            # agregate loss on all levels
            loss = torch.tensor(0.).to(device=device)
            for k in result.keys():
                output = result[k]
                gt = batch['labels'][:, val_ds.label_names.index(k)] # type:ignore
                loss += criterion(output, gt)
            
            epoch_val_loss.append(loss.item())
    print(f"Epoch-{epoch:03}: Lt:{np.mean(epoch_train_loss):.4f} Lv:{np.mean(epoch_val_loss):.4f}")
    print('\n')
    

Validation     : 100%|██████████| 26692/26692 [09:03<00:00, 49.09it/s]


Epoch-000: Lt:0.4103 Lv:5.7524




Validation     : 100%|██████████| 26692/26692 [09:07<00:00, 48.76it/s]


Epoch-001: Lt:0.3975 Lv:7.0679




Validation     : 100%|██████████| 26692/26692 [08:46<00:00, 50.73it/s]


Epoch-002: Lt:0.3930 Lv:7.6406




Training       :  48%|████▊     | 27543/57700 [13:06<14:20, 35.03it/s]  


KeyboardInterrupt: 

In [ ]:
# Training       : 100%|██████████| 1804/1804 [19:05<00:00,  1.57it/s]
# Validation     : 100%|██████████| 835/835 [07:57<00:00,  1.75it/s]
# Epoch-000: Lt:0.4091 Lv:1.4503


# Training       : 100%|██████████| 1804/1804 [19:28<00:00,  1.54it/s]
# Validation     : 100%|██████████| 835/835 [07:31<00:00,  1.85it/s]
# Epoch-001: Lt:0.3276 Lv:1.1352


# Training       : 100%|██████████| 1804/1804 [19:20<00:00,  1.55it/s]
# Validation     : 100%|██████████| 835/835 [07:06<00:00,  1.96it/s]
# Epoch-002: Lt:0.2901 Lv:1.0932


# Training       : 100%|██████████| 1804/1804 [19:16<00:00,  1.56it/s]
# Validation     : 100%|██████████| 835/835 [06:57<00:00,  2.00it/s]
# Epoch-003: Lt:0.2891 Lv:0.7242




In [ ]:
# visualize normals
# proj = train_ds.projects[7]
for i, proj in enumerate(train_ds.projects):
    PlyData([PlyElement.describe(pd.DataFrame(np.concatenate([proj['xyz'],proj['normals'],np.squeeze(proj['incAngles'])[:,None],np.proj['distanceFromScanner'][:,None], proj['sop_ids'][:,None]], axis=1),columns=['x','y','z','nx','ny','nz','incAngle','distanceFromScanner','sop_ids']).to_records(index=False),'vertex')]).write(f"./_temp/proj_{i:03}.ply")

In [ ]:
for arr in [proj['xyz'],proj['normals'],np.squeeze(proj['incAngles'])[:,None],proj['distanceFromScanner'][:,None], proj['sop_ids'][:,None]]:
    print(arr.shape)

In [138]:
np.set_printoptions(suppress=True)
train_ds[0]['features_neibors'].std(axis=0)

array([0.01236105, 0.0084599 , 0.00521252, 0.02295678, 0.02016745,
       0.0234277 , 0.19764559, 0.15461153, 0.00264833, 0.        ,
       0.00000009, 0.44791493, 0.09292256, 0.04890899, 0.09555608,
       0.11370969], dtype=float32)

In [64]:
np.set_printoptions(suppress=True)
train_ds[0]['features_neibors'].std(axis=0)

array([0.01326171, 0.00831641, 0.00541755, 0.09005746, 0.07818554,
       0.07874302, 0.19764559, 0.15461153, 0.00264833, 0.        ,
       0.00000009, 0.4461691 , 0.09488796, 0.05305372, 0.08529734,
       0.10812434], dtype=float32)

In [ ]:
proj['incAngles'].mean()

In [ ]:
proj['incAngles'].std()

In [ ]:
proj['distanceFromScanner'].mean()

In [ ]:
(proj['distanceFromScanner']-proj['distanceFromScanner'].mean()).std()